In [1]:
import numpy as np
import sys
from scipy.linalg import expm
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi
from IPython.display import display
from single_hopping import *
from double_hopping import *
from qiskit import transpile
import matplotlib.pyplot as plt


In [2]:
def construct_inter_controlled_single_hopping_qubit_matrix(i,j,k,t,N):
    y = np.array([[0, -1j], [1j, 0]])
    x = np.array([[0, 1], [1, 0]])
    z = np.array([[1, 0], [0, -1]])
    I = np.eye(2)
    A = 1
    B = 1
    C = 1
    D = 1
    for a in range(N):
        if a < i:
            A = np.kron(A,I)
            B = np.kron(B,I)
            C = np.kron(C,I)
            D = np.kron(D,I)
        elif a == i:
            A = np.kron(A,y)
            B = np.kron(B,y)
            C = np.kron(C,x)
            D = np.kron(D,x)
        elif a < j:
            A = np.kron(A,z)
            B = np.kron(B,z)
            C = np.kron(C,z)
            D = np.kron(D,z)
        elif a == j:
            A = np.kron(A,z)
            B = np.kron(B,I)
            C = np.kron(C,z)
            D = np.kron(D,I)
        elif a < k:
            A = np.kron(A,z)
            B = np.kron(B,z)
            C = np.kron(C,z)
            D = np.kron(D,z)
        elif a == k:
            A = np.kron(A,x)
            B = np.kron(B,x)
            C = np.kron(C,y)
            D = np.kron(D,y)
        else:
            A = np.kron(A,I)
            B = np.kron(B,I)
            C = np.kron(C,I)
            D = np.kron(D,I)
    return t*(-1j*A+1j*B+1j*C-1j*D)/4

In [3]:
N = 7
i = 0
j = 3
k = 4
t = 1
H = construct_inter_controlled_single_hopping_qubit_matrix(i, j, k, t, N)
ops = FermionicOp({"+_" + str(i)+" +_"+str(j)+ " -_" + str(j)+" -_"+str(k) : t,\
                    "+_" + str(k)+" +_"+str(j)+ " -_" + str(j)+" -_"+str(i):-t}, num_spin_orbitals=N)
mapper = JordanWignerMapper()
jw_op = mapper.map(ops)
jw_op = qi.Operator(jw_op).to_matrix()
jw_op = np.array(jw_op)

eigH = np.linalg.eig(H)[0]
eigJW = np.linalg.eig(jw_op)[0]
eigH = np.sort(1j*eigH)
eigJW = np.sort(1j*eigJW)
print(f"i = {i}, j = {j}, k = {k}, N = {N}")
print(f"The norm of the eigenvalue of qubit matrix is {np.linalg.norm(eigH)}")
print(f"The norm of the eigenvalue of the JW transformation from Qiskit is {np.linalg.norm(eigJW)}")
print(f"The difference between two eigenvalue vector is {np.sum(abs(eigH - eigJW))}")

i = 0, j = 3, k = 4, N = 7
The norm of the eigenvalue of qubit matrix is 5.65685424949238
The norm of the eigenvalue of the JW transformation from Qiskit is 5.65685424949238
The difference between two eigenvalue vector is 0.0
